# Regresión Logística
Este notebook está hecho para la práctica de regresión logística en Scikitlearn

Para regresión logísitca se requiere
* Variable dependiente debe ser categórica o numérica discreta
* Si es una regresión logística binaria el valor 1 en la variable dependiente debe ser la categoria deseada
* Solo variables significantes deben ser incluídas.
* Las variables independintes deben ser independintes entre ellas.

El dataset fue obtenido de UCI Machine Learning repository, y esta relacionado con campañas de marqueting (llamadas telefónicas) de una institución bancaria. El objetivo de la clasificación es prevenir si el cliente va o no a adquirir un CDT

En google colaboratory ingresar dentro de una celda
```bash
!pip install -q pydotplus
!apt-get install graphviz
```

Luego en el menu buscar Entorno de ejecución -> Reiniciar entorno de ejecución

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

## Dataset
* Cargar el Dataset https://raw.githubusercontent.com/javiercocu/intro-datascience/master/data/bank-additional.csv en un DataFrame con la funcion pd.read_csv()
* La descripción del dataset se encuentra en: https://archive.ics.uci.edu/ml/datasets/bank+marketing

In [5]:
df_banking = pd.read_csv("https://raw.githubusercontent.com/javiercocu/intro-datascience/master/data/bank-additional.csv", sep=";")
df_banking.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
df_banking.keys()

# Ejercicio
Realizar Análisis inicial y estadística descriptiva,
* Explore los datos, valores únicos por columna con la funcion unique()
* Verificar si hay datos faltantes
* Qué se debería hacer con los datos faltantes?
* Qué tipo de variables estadísticas son cada una de las características
* Use la funcion df_boston.describe() para analizar los principales datos de estadística descriptiva del DataSet
* Cree gráficos de histogramas
* Cree un gráfico de la matriz de correlación del DataFrame
    * ¿Cuáles de las características tienen alta correlación entre ellas?
* ¿Cuáles de las caractarísticas tienen más relación con la variable objetivo según diagramas de dispersión?

* Incluir dentro del análisis visual para las variables categóricas ('job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome') un gráfico de barras contra variable objetivo

```python
pd.crosstab(df_banking.job,df_banking.y).plot(kind='bar')
plt.title('CDT abiertos por trabajo')
plt.xlabel('Trabajo')
plt.ylabel('Frecuencia de CDT abiertos')```

In [ ]:
#Celdas para completar ejercicio








In [ ]:
df_banking["education"].unique()

In [ ]:
#Agrupamiento de valores
dic_reemplazo = {'basic.4y':'basic', 'high.school':'basic', 'basic.6y':'basic', 'basic.9y':'basic'}

df_banking['education'] = df_banking['education'].replace(dic_reemplazo)
df_banking["education"].unique()

In [ ]:
# Frecuencias de la variable objetivo
df_banking['y'].value_counts()

In [ ]:
# Agrupar por variable objetivo para descubrir posibles relaciones
df_banking.groupby('y').mean()

In [ ]:
### Qué se observa de esta agrupación contra la variable objetivo?



In [ ]:
# Convertir variables categóricas en uno a muchos

# Variables categóricas
variables_cat=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']

for var_cat in variables_cat:
    df_banking2 = df_banking.join(pd.get_dummies(df_banking[var_cat], prefix = var_cat))
    df_banking = df_banking2
df_banking.keys()


In [ ]:
df_banking.head()

In [ ]:
lista_columnas = df_banking.columns.tolist()
lista_columnas

In [ ]:
columnas_sin_cat = []
for columna in lista_columnas:
    if columna not in variables_cat:
        columnas_sin_cat.append(columna)
        
columnas_sin_cat

In [ ]:
df_banking_final = df_banking[columnas_sin_cat]
df_banking_final.columns

In [ ]:
# transformación variable objetivo en binario
df_banking_final['y'] = df_banking_final['y'].map({'no':0,'yes':1})

In [ ]:
# Valores unicos en variable dependiente
df_banking_final['y'].unique()

In [ ]:
# Separar datasets de variables independientes X, y variable objetivo Y

X = df_banking_final.drop('y', axis=1)
X.columns

In [ ]:
y = df_banking_final['y']
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('La precisión del modelo de regresión logística en el set de test es:', logreg.score(X_test, y_test))

### Validación cruzada 
Trata de prevenir el sobreajuste (overfitting) mientras se entrena con todo el dataset.
En este caso usaremos 10 divisiones sobre el dataset.

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("Promedio de precisión sobre validación cruazada: %.3f" % (results.mean()))

Como el promedio de precisión sobre la validación cruzada es cercano a la precisión del modelo, esto quiere decir que el modelo generaliza bien

### Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

### Precisión y exhaustividad

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

La precisión y exhaustividad (denominado a veces como exhaustividad y precisión) es una métrica empleada en la medida del rendimiento de los sistemas de búsqueda y recuperación de información y reconocimiento de patrones. En este contexto se denomina precisión (denominado igualmente valor positivo predicho) como a la fracción de instancias recuperadas que son relevantes, mientras recall (denominado igualmente sensibilidad o exhaustividad) es la fracción de instancias relevantes que han sido recuperadas.1 Tanto la precisión como la exhaustividad son entendidas como medidas de la relevancia. Para entender mejor el concepto, supongamos de la existencia de un programa que reconoce perros en fotografías, dicho programa reconoce 7 perros en una escena que contiene 9 perros y algunos gatos. Si 4 de las identificaciones han sido correctas, pero 3 eran gatos, el programa tendrá una precisión de 4/7 mientras que posee una sensibilidad de 4/9. Otro ejemplo en el que participa un motor de búsqueda que, ante una consulta dada, retorna 30 páginas de las cuales sólo 20 son relevantes dejando 40 páginas relevantes fuera de la búsqueda. Este motor tendrá entonces una precisión de 20/30 = 2/3 mientras que su sensibilidad es 20/60 = 1/3.

Para un usuario la situación ideal es aquella en la que existe una precisión y exhaustividad alta (es decir muy cercana a 1). A esta situación se la denomina utilidad teórica. Con el objeto de ponderar y ver cual lejano se encuentran ambas medidas del la utilidad teórica, suele emplearse los valores de ambas métricas combinadas en una media armónica denominada valor-F.

(Extraído de wikipedia)

## Ejemplo 2
### Regresión logística - reconocimiento de dígitos

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
# Forma del dataset independientes
print("Image Data Shape" , digits.data.shape)

#Forma del dataset variable objetivo
print("Digitos", digits.target.shape)

In [ ]:
# Visualización de dígitos
import numpy as np 
import matplotlib.pyplot as plt
plt.figure(figsize=(20,4))
for index, (image, label) in enumerate(zip(digits.data[0:5], digits.target[0:5])):
 plt.subplot(1, 5, index + 1)
 plt.imshow(np.reshape(image, (8,8)), cmap=plt.cm.gray)
 plt.title('Training: %i\n' % label, fontsize = 20)

In [ ]:
#Dividir datos de entrenamiento y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

In [ ]:
#Entrenar modelo
logisticRegr.fit(x_train, y_train)

In [ ]:
# Predecir valores no vistos
logisticRegr.predict(x_test[0].reshape(1,-1))

In [ ]:
# Predecir múltiples valores
logisticRegr.predict(x_test[0:10])

In [ ]:
predictions = logisticRegr.predict(x_test)

In [ ]:
# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [ ]:
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(cm, interpolation='nearest', cmap='Pastel1')
plt.title('Confusion matrix', size = 15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], rotation=45, size = 10)
plt.yticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], size = 10)
plt.tight_layout()
plt.ylabel('Actual label', size = 15)
plt.xlabel('Predicted label', size = 15)
width, height = cm.shape
for x in iter(range(width)):
 for y in iter(range(height)):
  plt.annotate(str(cm[x][y]), xy=(y, x), 
  horizontalalignment='center',
  verticalalignment='center')

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
index = 0
misclassifiedIndexes = []
for label, predict in zip(y_test, prediction):
 if label != predict: 
  misclassifiedIndexes.append(index)
  index +=1